<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_infinity_synthesis_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_infinity_synthesis_ai.py

Full pipeline for InfinitySynthesisAI:
1. Synthetic “infinite structure” dataset of 6 inputs → 3 targets
2. float32 normalization & dtype consistency
3. Model with Linear→ReLU→Linear
4. Physics‐informed residual enforcing toy “infinite recursion” laws
5. MC‐Dropout for uncertainty quantification
6. Training loop: AdamW, ReduceLROnPlateau, grad clipping, NaN checks, early stopping
7. Checkpointing & safe reload
8. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Infinity Synthesis Dataset
# ------------------------------------------------------------------------------
class InfinityDataset(Dataset):
    def __init__(self, n_samples=6000, seed=42):
        np.random.seed(seed)
        # Features:
        # HDD: hyper-structure density ∈ [0.1, 10.0]
        # RIF: recursive infinity factor ∈ [1, 100]
        # OEF: omniversal expansion ∈ [0.01, 1.0]
        # TDF: time dilation factor ∈ [0.5, 2.0]
        # QSB: quantum singularity balance ∈ [0.1, 5.0]
        # ENF: entropy nesting factor ∈ [0.01, 2.0]
        HDD = np.random.uniform(0.1, 10.0,   (n_samples,1))
        RIF = np.random.uniform(1.0, 100.0,  (n_samples,1))
        OEF = np.random.uniform(0.01, 1.0,   (n_samples,1))
        TDF = np.random.uniform(0.5, 2.0,    (n_samples,1))
        QSB = np.random.uniform(0.1, 5.0,    (n_samples,1))
        ENF = np.random.uniform(0.01,2.0,    (n_samples,1))

        X_raw = np.hstack([HDD, RIF, OEF, TDF, QSB, ENF]).astype(np.float64)

        # Toy “infinite recursion” targets:
        # IRD: recursion depth = RIF * OEF / (ENF + eps)
        # STR: stability = HDD * TDF * QSB
        # EFF: loop efficiency = log1p(RIF) * OEF
        eps = 1e-6
        IRD = RIF * OEF / (ENF + eps)
        STR = HDD * TDF * QSB
        EFF = np.log1p(RIF) * OEF

        Y_raw = np.hstack([IRD, STR, EFF]).astype(np.float64)
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-8
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-8

        # Normalize to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )

# ------------------------------------------------------------------------------
# 2. InfinitySynthesisAI Model
# ------------------------------------------------------------------------------
class InfinitySynthesisAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3):
        super().__init__()
        self.fc1  = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2  = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# ------------------------------------------------------------------------------
# 3. Physics‐Informed Residual Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    HDD, RIF, OEF, TDF, QSB, ENF = X_den.t()
    eps = 1e-6

    IRD_t = RIF * OEF / (ENF + eps)
    STR_t = HDD * TDF * QSB
    EFF_t = torch.log1p(RIF) * OEF

    Yt = torch.stack([IRD_t, STR_t, EFF_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

# ------------------------------------------------------------------------------
# 4. Combined Loss
# ------------------------------------------------------------------------------
def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 5. MC‐Dropout Uncertainty
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, dim=0)
    return arr.mean(dim=0), arr.std(dim=0)

# ------------------------------------------------------------------------------
# 6. Training Loop & Checkpointing
# ------------------------------------------------------------------------------
def train(model, tr_loader, va_loader, stats, device,
          lr=1e-4, wd=1e-5, lam=1.0,
          epochs=100, patience=10):
    model.to(device)
    opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(
        opt, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for ep in range(1, epochs+1):
        # Train
        model.train()
        run_tr = 0.0
        for Xb, Yb in tr_loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats, lam)
            if torch.isnan(loss):
                print(f"NaN at epoch {ep}, abort.")
                return history
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            run_tr += loss.item() * Xb.size(0)
        tr_loss = run_tr / len(tr_loader.dataset)

        # Validate
        model.eval()
        run_va = 0.0
        with torch.no_grad():
            for Xb, Yb in va_loader:
                Xb, Yb = Xb.to(device), Yb.to(device)
                pred = model(Xb)
                l, _, _ = total_loss(pred, Yb, Xb, stats, lam)
                run_va += l.item() * Xb.size(0)
        va_loss = run_va / len(va_loader.dataset)

        sched.step(va_loss)
        history['train'].append(tr_loss)
        history['val'].append(va_loss)
        print(f"Epoch {ep:03d} | Train {tr_loss:.4e} | Val {va_loss:.4e}")

        if va_loss < best_val - 1e-6:
            best_val, wait = va_loss, 0
            torch.save(model.state_dict(), "best_infinity_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    # Load best
    if os.path.exists("best_infinity_ai.pth"):
        model.load_state_dict(torch.load(
            "best_infinity_ai.pth", map_location=device))
    return history

# ------------------------------------------------------------------------------
# 7. Visualizations
# ------------------------------------------------------------------------------
def plot_history(history):
    plt.figure()
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.show()

def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=6, alpha=0.5)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.title(title); plt.show()

def plot_uncertainty(model, stats, device):
    G = 100
    RIF = np.linspace(1.0,100.0,G, dtype=np.float32)
    OEF = np.linspace(0.01,1.0, G, dtype=np.float32)
    G1, G2 = np.meshgrid(RIF, OEF)
    pts = G*G

    Xg = torch.zeros((pts,6), device=device, dtype=torch.float32)
    # fix HDD, TDF, QSB, ENF at mean
    Xg[:,0] = stats['X_mean'][0]
    Xg[:,2] = stats['X_mean'][2]
    Xg[:,3] = stats['X_mean'][3]
    Xg[:,4] = stats['X_mean'][4]
    Xg[:,5] = stats['X_mean'][5]
    Xg[:,1] = torch.from_numpy(G1.ravel()).to(device)
    Xg[:,2] = torch.from_numpy(G2.ravel()).to(device)

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=50)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(RIF, OEF, U, shading='auto', cmap='magma')
    plt.colorbar(label="Std(IRD)")
    plt.xlabel("Recursive Infinity Factor (RIF)")
    plt.ylabel("Omniversal Expansion (OEF)")
    plt.title("Uncertainty: Recursion Depth")
    plt.show()

# ------------------------------------------------------------------------------
# 8. Main
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds    = InfinityDataset(n_samples=6000, seed=42)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  dtype=torch.float32, device=device),
    }

    n_val = int(0.2 * len(ds))
    tr_ds, va_ds = random_split(ds, [len(ds)-n_val, n_val])
    tr_ld = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_ld = DataLoader(va_ds, batch_size=256, shuffle=False)

    model   = InfinitySynthesisAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        Yp_n = model(X_all).cpu().numpy()
    Yt = ds.Y * ds.Y_std + ds.Y_mean
    Yp = Yp_n * ds.Y_std + ds.Y_mean

    names = ["Recursion Depth","Stability","Loop Efficiency"]
    for i, nm in enumerate(names):
        plot_scatter(Yt[:,i], Yp[:,i], nm)

    plot_uncertainty(model, stats, device)